# Deep-Learning-from-Scratch
## Chap6. Skills that related to Learning

이번 장에서는 신경망 학습의 핵심 개념들을 만나본다.<br/>
* 이번 장에서 만나볼 친구들
    * 가중치 매개변수의 최적값을 탐색하는 최적화 방법
    * 가중치 매개변수의 초깃값
    * 하이퍼파라미터 설정 방법
    * 이 외에 오버피팅의 대응책인 가중치 감소, 드롭 아웃 등의 정규화 방법들도 살펴본다.
    * 배치정규화도 살펴본다

이러한 방법들을 사용하면 신경망(딥러닝) 학습의 효율과 정확도를 높일 수 있다

이전에도 배웠던 적이 있지만 조금 쉬쉬하고 넘겼던 파트들이다...<br/>
이번기회에 확실히 개념을 다져두자!

## 6.1 매개변수 갱신

신경망 학습의 목적은 손실 함수의 값을 가능한 한 낮추는 매개변수를 찾는 것이다<br/>
이는 곧 매개변수의 최적값을 찾는 문제이며, 이러한 문제를 푸는 것으르 **최적화(Optimizatino)**라 한다.

안타깝게도 이러한 초최적화는 굉장히 어려운 문제이다(손실 함수의 값을 최대한 낮추는 매개변수를 찾는 과정은 멀고도 험해!)<br/>
이는 매개 변수 공간이 매우 넓고 복잡해서 최적의 솔루션을 쉽게 찾지 못하는데 있다. 수식을 풀어 순식간에 최적의 솔루션을 찾는 방법은 없다. 무엇보다도 심층 신경망에서는 매개변수의 수가 엄청나게 많아져서 사태는 더욱 심각해진다

지금까지는 SGD(확률적 경사하강법)방법을 사용했다. <br/>
매개변수의 기울기를 구해, 기울어진 방향으로 매개변수 값을 갱신하는 일을 몇 번이고 반복해서 점점 최적의 값을 찾아나갔다. <br/>
그런데 문제에 따라서는 SGD보다 똑똑한 방법들이 있다. 지금부터는 그러한 방법들에 대해 알아본 후, SGD와는 다른 최적화 기법들을 살펴보자!

### 6.1.1 모험가 이야기

"전설에 나오는 세상에서 가장 깊고 낮은 골짜기, 그곳에 내 모든 보물을 두고왔다"<br/>
모험가는 2가지 제약조건을 가지고 깊은 골짜기를 찾아 나선다.<br/>
하나, 지도를 보지 않을 것<br/>
둘, 눈가리개를 사용할 것<br/>
오직 발바닥으로 전해지는 기울기를 통해 지금 있는 장소에서 가장 기울어진 방향으로 내려가는 것이 바로 모험가의 전략이다.<br/>
이는 **SGD**가 취하는 전략과 같다.

### 6.1.2 확률적 경사 하강법(SGD)

In [4]:
# SGD 구현하기
# 나중에 사용할 것을 생각해 클래스의 이름을 SGD라 명명한다.
# Again Revuew SGD - 이제 수식의 의미를 완전히 이해할 수 있겠다!

class SGD:
    def __init__(self, lr=0.01):
        self.lr = lr
        
    def update(self, params, grads):
        for key in params.keys():  # params까지는 알겠는데... params.key()는 무엇을 의미하는거지?
            params[key] -= self.lr * grads[key]

1. 초기화 때 받는 인수 'lr'은 learning rate(학습률)을 뜻한다. <br/>
이 학습률을 인스턴스 변수로 유지한다. <br/>
<br/>
2. update(params, grads) 메서드는 SGD과정에서 반복해서 호출된다. <br/>
인수인 params와 grads는 (지금까지 구현했던 것과 마찬가지로) 딕셔너리 변수이다.<br/>
params['W1'], grads['W1']등과 같이 각각 가중치 매개변수와 기울기를 저장하고 있다. <br/>
딕셔너리 변수의 key값을 하나씩 읽어오는거구나...! params에서 key를 불러오면 매개변수 하나하나 불러올 수 있겠고, 이는 파라미터 업데이트에 사용되겠군!

In [5]:
# 이런식으로 코드가 작동할 것이다

network = TwoLayerNet(...)   <br>
optimizer = SGD()            <br>
<br>
for i in range(10000):  # epoch은 10000<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;'''<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;x_batch, t_batch = get_mini_batch(...)   # 미니배치<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;grads = network.gradient(x_batch, t_batch)<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;params = network.params<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;optimizer.update(params, grads)<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;'''

네트워크 불러오고, optimizer 지정해주고, epoch만큼 반복하면서 미니배치 학습한다. <br/>
optimizer는 최적화를 행하는 자라는 단어인데 이 코드에서는 SGD가 그 역할을 한다<br/>
매개변수 갱신은 optimizer가 책임지고 수행하니 우리는 optimizer에 매개변수(params)와 기울기 정보(grads)만 넘겨주면 된다<br/>
이처럼 최적화를 담당하는 클래스를 분리해 구현하면 기능을 모듈화 하기 좋다.

### 6.1.3 SGD의 단점

SGD는 단순하고 구현도 쉽지만, 문제에 따라서는 비효율적일 때가 있다.<br/>
이번 절에서는 SGD의 단점을 알아보고자 다음 함수의 최솟값을 구하는 문제를 생각해보겠다.<br/>
f(x, y) = (1/20*x**2) + y**2

* 참고 - 잘 정리된 블로그 
    * 수식과 코드로 보는 경사하강법
    * https://twinw.tistory.com/247

### 6.1.4 모멘텀

In [7]:
import numpy as np

class Momentum:
    def __init__(self, lr=0.01, momentum=0.9):
        self.lr = lr
        self.momentum = momentum
        self.v = None  # 인스턴스 변수 v는 물체의 속도이다.
                       # 초기화시 아무 값도 담지 않는다.
    
    def update(self, params, grads):
        if self.v is None:
            self.v = {}  
            for key, val in params.items():
                self.v[key] = np.zeros_likeke(val)  # 인스턴스 변수v는 update()가 처음 호출될 때
                                                    # 매개변수와 같은 구조의 데이터를 딕셔너리 변수로 저장한다.
            for key in params.keys():
                self.v[key] = self.momentum*self.momentum*self.v[key] - self.lr*grads[key]
                params[key] += self.v[key]

### 6.1.5 AdaGrad

신경망 학습에서는 학습률(수식에서는 에타로 표기) 값이 중요하다.<br/>
이 값이 너무 작으면 학습 시간이 너무 길어지고, 반대로 너무 크면 발산하여 학습이 제대로 이뤄지지 않는다.<br/>
이 학습률을 정하는 효과적 기술로 **학습률 감소(learning rate decay)**가 있다.** 이는 학습을 진행하면서 학습률을 점차 줄여가는 방법이다.<br/>
처음에는 크게 학습하다가 조금씩 작게 학습한다는 얘기로, 실제 신경망 학습에 자주 쓰인다.

### 6.1.6 Adam

### 6.1.7 어느 갱신 방법을 이용할 것인가?

### 6.1.8 MNIST 데이터셋으로 본 갱신 방법 비교

## 6.2 가중치의 초깃값

### 6.2.1 초깃값을 0으로 하면?

### 6.2.2 은닉층의 활성화값 분포

### 6.2.3 ReLU를 사용할 때의 가중치 초깃값

### 6.2.4 MNIST 데이터셋으로 본 가중치 초깃값 비교

## 6.3 배치 정규화

### 6.3.1 배치 정규화 알고리즘

하나하나 채워나가보자 20.02.06.Thur pm12:13